### Needed package

In [1]:
import pandas as pd
import numpy as np
import nested_dict as nd
from pyscipopt import Model

### Data Definition:
For example:

* dem_df:
    * the amount of order of 90A is 800
    * the amount of order of 90B is 500
    * all of the SKU are with different priority. the lower the number is, the priorer the SKU is.
* bom_df:
    * 90A is composed of one Comp1 and one Comp2
    * 90B is composed of one Comp1, one Comp2 and one Comp3
* con_df:
    * the assignable amount of 1_1 is 1000 and 1_2 is 800, and both of them can be Comp1
* sub_df:
    * describe the relationship between sku and substitute
    * priority=1 means main meterial and priority=2 means substitute

In [2]:
CRED = '\033[91m'
CEND = '\033[0m'

all_df = pd.read_excel('./Data/example_scip.xlsx', sheet_name=None)
dem_df = all_df['SO']
bom_df = all_df['BOM']
con_df = all_df['constraints']
con_df = con_df[con_df.con_qty!=0]
sub_df = all_df['substitute']
sub_df = sub_df.fillna(method='ffill')
sub_df = sub_df[(sub_df.priority!=0)&(sub_df.substitute.isin(con_df.substitute))]
bom_dict = bom_df.groupby(['down']).apply(lambda x:x.set_index('SKU')['qty'].to_dict()).reset_index()
bom_dict = bom_dict.set_index('down')[0].to_dict()
con_dict = con_df.set_index('substitute')['con_qty'].to_dict()
dem_dict = dem_df.set_index('SKU')['Demand'].to_dict()

print(CRED+'dem_df:'+CEND, dem_df, sep='\n')
print(CRED+'bom_df:'+CEND, bom_df, sep='\n')
print(CRED+'con_df:'+CEND, con_df, sep='\n')
print(CRED+'sub_df:'+CEND, sub_df, sep='\n')

dem_df:
   SKU  Demand  Priority
0  90A     800         1
1  90B     500         1
2  90C     700         1
3  90D     400         2
4  90E     600         2
5  90F     700         3
6  90G    1000         3
bom_df:
    SKU   down  qty
0   90A  Comp1    1
1   90A  Comp2    1
2   90B  Comp1    1
3   90B  Comp2    1
4   90B  Comp3    1
5   90C  Comp2    1
6   90C  Comp3    1
7   90D  Comp1    1
8   90E  Comp1    1
9   90E  Comp2    1
10  90F  Comp1    1
11  90F  Comp3    1
12  90G  Comp1    1
13  90G  Comp2    1
con_df:
    down substitute  con_qty
0  Comp1        1_1     1000
1  Comp1        1_2      800
2  Comp2        2_1      500
3  Comp2        2_2      300
4  Comp3        3_1      200
sub_df:
    SKU   down substitute  priority
0   90A  Comp1        1_1         1
2   90A  Comp2        2_1         1
6   90B  Comp1        1_1         1
7   90B  Comp1        1_2         2
8   90B  Comp2        2_1         2
9   90B  Comp2        2_2         1
10  90B  Comp3        3_1         1
14  90

### Resources about preemptive goal programming:
* [Goal Programming](http://du.ac.in/du/uploads/departments/Operational%20Research/25042020_Goal%20Programming.pdf)
* [Preemptive Goal Programming](https://www.riverware.org/PDF/RiverWare/documentation/80_prerelease/index.html#page/Optimization/OptimMath.3.1.html)
* [Preemptive Goal Programming example in pulp](https://github.com/yuning-lin/SideProjects/blob/main/LinearProgramming/pulp_with_preemptive_goal_programming.ipynb)

In [24]:
# model building
model = Model("Minimum Delay Shipment")  # model name is optional
model.hideOutput() # not print output

# create unknown parameters
z = nd.nested_dict()
for idx, row in dem_df.iterrows():
    z[row['SKU']] = model.addVar(vtype="B", name=f"z_{row['SKU']}")
d = nd.nested_dict()
for idx, row in sub_df.iterrows():
    d[row['substitute']][row['down']][row['SKU']] = model.addVar(vtype="I", lb=0, ub=dem_dict[row['SKU']], name=f"dem_{row['SKU']}_{row['down']}_{row['substitute']}")

### Equations
* objective: $min_{Z_{i}}\sum_{i}Z_{i}*DEM_{i}$
    * minimum the amount of delayed shipment
* equation 1: $\sum_{i}Z_{i}*D_{ikl}*BOM_{ik}=CON_{kl}$
    * 所有無法出貨的需求量須和缺料量相等
* equation 2: $(1-Z_{i})*D_{ikl}=0$
    * 若有出貨的單 $D_{ikl}$ 須為 0；反之 $Z_{i}$ 須為 1
* equation 3: $\sum_{l\in L_{ik}}D_{ikl} \leq DEM_{i}$
    * 產品 i 中因為缺少 l 作為 k 的替料，而無法生產的產品數需 $\leq$ 產品 i 的需求數
* equation 4: $(\sum_{i\in P^{1}_{kl}}DEM_{i}*BOM_{ik}-\sum_{i\in P^{1}_{kl}}Z_{i}*D_{ikl}*BOM_{ik})*\sum_{i \in P^{2}_{kl}}Z_{i}*D_{ikl}*BOM_{ik}=0$
    * 括弧中代表主料、非括弧項表替料，此式為主料沒用完前不會用替料
    

In [25]:
#### equation 1:
for key, group in sub_df.groupby('substitute'):
    print(sum([z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']] for _, row in group.iterrows()])==con_dict[key])
    model.addCons(sum([z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']] for _, row in group.iterrows()])==con_dict[key])

#### equation 2:
for key, group in sub_df.groupby('substitute'):
    for _, row in group.iterrows():
        print((1-z[row['SKU']])*d[key][row['down']][row['SKU']]==0)
        model.addCons((1-z[row['SKU']])*d[key][row['down']][row['SKU']]==0)

#### equation 3:
for key, group in sub_df.groupby(['down','SKU']):
    print(sum([d[row['substitute']][key[0]][key[1]] for _, row in group.iterrows()])<=dem_dict[key[1]])
    model.addCons(sum([d[row['substitute']][key[0]][key[1]] for _, row in group.iterrows()])<=dem_dict[key[1]])

#### equation 4:
for key, group in sub_df.groupby('substitute'):
    main_set = sub_df[(sub_df.priority==1)&(sub_df.substitute==key)]
    subs_set = sub_df[(sub_df.priority==2)&(sub_df.substitute==key)]
    print(main_set, subs_set, sep='\n')
    print(sum([dem_dict[row['SKU']]*bom_dict[row['down']][row['SKU']]\
                        -z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']] for _, row in main_set.iterrows()])\
                        *sum([z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']]for _, row in subs_set.iterrows()])==0)
    model.addCons(sum([dem_dict[row['SKU']]*bom_dict[row['down']][row['SKU']]\
                       -z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']] for _, row in main_set.iterrows()])\
                       *sum([z[row['SKU']]*d[key][row['down']][row['SKU']]*bom_dict[row['down']][row['SKU']]for _, row in subs_set.iterrows()])==0)


ExprCons(Expr({Term(z_90A, dem_90A_Comp1_1_1): 1.0, Term(z_90B, dem_90B_Comp1_1_1): 1.0, Term(dem_90D_Comp1_1_1, z_90D): 1.0, Term(dem_90E_Comp1_1_1, z_90E): 1.0, Term(dem_90G_Comp1_1_1, z_90G): 1.0}), 1000.0, 1000.0)
ExprCons(Expr({Term(z_90B, dem_90B_Comp1_1_2): 1.0, Term(dem_90D_Comp1_1_2, z_90D): 1.0, Term(dem_90F_Comp1_1_2, z_90F): 1.0, Term(dem_90G_Comp1_1_2, z_90G): 1.0}), 800.0, 800.0)
ExprCons(Expr({Term(z_90A, dem_90A_Comp2_2_1): 1.0, Term(z_90B, dem_90B_Comp2_2_1): 1.0, Term(dem_90C_Comp2_2_1, z_90C): 1.0, Term(dem_90E_Comp2_2_1, z_90E): 1.0, Term(dem_90G_Comp2_2_1, z_90G): 1.0}), 500.0, 500.0)
ExprCons(Expr({Term(z_90B, dem_90B_Comp2_2_2): 1.0, Term(dem_90C_Comp2_2_2, z_90C): 1.0}), 300.0, 300.0)
ExprCons(Expr({Term(z_90B, dem_90B_Comp3_3_1): 1.0, Term(dem_90C_Comp3_3_1, z_90C): 1.0, Term(dem_90F_Comp3_3_1, z_90F): 1.0}), 200.0, 200.0)
ExprCons(Expr({Term(z_90A, dem_90A_Comp1_1_1): -1.0, Term(dem_90A_Comp1_1_1): 1.0}), 0.0, 0.0)
ExprCons(Expr({Term(z_90B, dem_90B_Comp1_1_1)

### Add objective with preemptive goal programming

In [18]:
#Objective
for p in dem_df.Priority.unique():
    print(f'Priority:{p}')
    pri_sku = dem_df[dem_df.Priority==p].SKU.unique()
    model.setObjective(sum([z[sku]*dem_dict[sku] for sku, _ in dem_dict.items() if sku in pri_sku]), "minimize")

    #Solve
    model.optimize()
    assert model.getStatus() == 'optimal'
    obj_val = model.getObjVal()

    if p!=3:
        model.freeTransform() # avoid (Exception: SCIP: method cannot be called at this time in solution process!)
        model.addCons(sum([z[sku]*dem_dict[sku] for sku, _ in dem_dict.items() if sku in pri_sku])<=obj_val)

Priority:1
Priority:2
Priority:3


### Get solution with readable form

In [20]:
# get solution
z_df = pd.DataFrame.from_dict({key:model.getVal(z[key])
                               for key, _ in z.items()}, orient='index').reset_index()
z_df = z_df.rename(columns={'index':'SKU', 0:'delay_shipment'})
z_df['delay_shipment'] = z_df['delay_shipment'].astype('int')

d_df = pd.DataFrame.from_dict({
                                (row['substitute'], row['down'], row['SKU']): model.getVal(d[row['substitute']][row['down']][row['SKU']])
                                for idx, row in sub_df.iterrows()},
                                orient = 'index')
d_df['substitute'] = d_df.index.map(lambda x:x[0])
d_df['down'] = d_df.index.map(lambda x:x[1])
d_df['SKU'] = d_df.index.map(lambda x:x[2])
d_df['distributed_qty'] = d_df[0].astype('int')
d_df = d_df.reset_index(drop=True)
d_df = d_df.drop(columns=[0])
final_df = pd.merge(d_df, z_df, on='SKU', how='left')
final_df.distributed_qty *= final_df.delay_shipment

pd.pivot_table(d_df, values='distributed_qty', index=['SKU'], columns=['substitute'], aggfunc=np.sum, fill_value=np.nan)

substitute,1_1,1_2,2_1,2_2,3_1
SKU,,,,,
90A,0.0,NaN,0.0,NaN,NaN
90B,0.0,100.0,0.0,300.0,200.0
90C,NaN,NaN,0.0,0.0,0.0
90D,0.0,0.0,NaN,NaN,NaN
90E,0.0,NaN,0.0,NaN,NaN
90F,NaN,700.0,NaN,NaN,0.0
90G,1000.0,0.0,500.0,NaN,NaN
